This script is what created the dataset pickled.

1) You need to download this file and put it in the same directory as this file.
https://github.com/moses-smt/mosesdecoder/raw/master/scripts/tokenizer/tokenizer.perl . Give it execution permission.

2) Get the dataset from http://ai.stanford.edu/~amaas/data/sentiment/ and extract it in the current directory.

3) Then run this script.

In [1]:
"""

"""

path='/home/jorge/data/text/sentiment/'

import numpy
import cPickle as pkl

from collections import OrderedDict

import glob
import os

from subprocess import Popen, PIPE

# tokenizer.perl is from Moses: https://github.com/moses-smt/mosesdecoder/tree/master/scripts/tokenizer
tokenizer_cmd = ['./tokenizer.perl', '-l', 'en', '-q', '-']



In [2]:
#Save text as CSV
import pandas as pd


with open(path+'rt-polarity.neg') as f:
    neg = f.read().splitlines()

with open(path+'rt-polarity.pos') as f:
    pos = f.read().splitlines()

sentences = pos + neg

print len(sentences)

y = [1] * len(pos) + [0] * len(neg) 

print len(y)

10662
10662


In [1]:
def tokenize(sentences):

    print 'Tokenizing..',
    text = "\n".join(sentences)
    tokenizer = Popen(tokenizer_cmd, stdin=PIPE, stdout=PIPE)
    tok_text, _ = tokenizer.communicate(text)
    toks = tok_text.split('\n')[:-1]
    print 'Done'

    return toks

print tokenize(sentences)[0]

NameError: name 'sentences' is not defined

In [4]:
#create dictionary with most frequent words first
def build_dict(sentences):

    #sentences = tokenize(sentences)

    print 'Building dictionary..',
    wordcount = dict()
    for ss in sentences:
        words = ss.strip().lower().split()
        for w in words:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = wordcount.values()
    keys = wordcount.keys()

    sorted_idx = numpy.argsort(counts)[::-1]

    worddict = dict()

    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+2  # leave 0 and 1 (UNK)

    print numpy.sum(counts), ' total words ', len(keys), ' unique words'

    return worddict


In [5]:
def grab_data(sentences, dictionary):
    #sentences = tokenize(sentences)

    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        words = ss.strip().lower().split()
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in words]

    return seqs

In [6]:
# Get the dataset from http://ai.stanford.edu/~amaas/data/sentiment/
dictionary = build_dict(sentences)
X = grab_data(sentences, dictionary)


Building dictionary.. 224067  total words  21425  unique words


In [7]:
print len(X)
print X[0]

10662
[3, 663, 9, 2654, 8, 23, 3, 3523, 16807, 102, 37, 7929, 37, 6, 11, 326, 255, 8, 70, 5, 3918, 56, 2966, 36, 1521, 2365, 4, 16100, 1718, 8090, 44, 914, 13648, 2]


In [8]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
'''Train a LSTM on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF+LogReg.
Notes:
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
GPU command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python imdb_lstm.py
'''

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import imdb

max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)



Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled)


In [10]:
#Max features
def remove_features(x):
    return [[0 if w >= max_features else w for w in sen] for sen in x]

X_train = remove_features(X_train)
X_test  = remove_features(X_test)


In [11]:
print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)



Pad sequences (samples x time)
('X_train shape:', (7143, 100))
('X_test shape:', (3519, 100))


In [19]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(512, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              class_mode="binary")


Build model...


In [20]:
batch_size = 128

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test), show_accuracy=True)


Train...
Train on 7143 samples, validate on 3519 samples
Epoch 1/15
7143/7143 [==============================] - 26s - loss: 0.7061 - acc: 0.5071 - val_loss: 0.6760 - val_acc: 0.6263
Epoch 2/15
7143/7143 [==============================] - 26s - loss: 0.5955 - acc: 0.6935 - val_loss: 0.5362 - val_acc: 0.7343
Epoch 3/15
7143/7143 [==============================] - 28s - loss: 0.2582 - acc: 0.8995 - val_loss: 0.6092 - val_acc: 0.7482
Epoch 4/15
7143/7143 [==============================] - 29s - loss: 0.0719 - acc: 0.9761 - val_loss: 1.0792 - val_acc: 0.7084
Epoch 5/15
7143/7143 [==============================] - 29s - loss: 0.0457 - acc: 0.9847 - val_loss: 1.0100 - val_acc: 0.7161
Epoch 6/15
7143/7143 [==============================] - 29s - loss: 0.0179 - acc: 0.9952 - val_loss: 1.2550 - val_acc: 0.7315
Epoch 7/15
7143/7143 [==============================] - 29s - loss: 0.0061 - acc: 0.9983 - val_loss: 1.4506 - val_acc: 0.7292
Epoch 8/15
7143/7143 [==============================] - 29s -

KeyboardInterrupt: 

In [21]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

3519/3519 [==============================] - 3s     
('Test score:', 1.7329132459013934)
('Test accuracy:', 0.73174197215117931)


In [22]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, model.predict_proba(X_test))


3519/3519 [==============================] - 3s     
0.800760168477
